In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 15)

In [129]:
df = pd.read_csv("books.csv", encoding="ISO-8859-1")

# DATA CLEANING

In [130]:
df.shape

(11127, 13)

In [131]:
df.head(20)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,err
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPrÃ©,4.57,439785960,9.78E+12,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPrÃ©,4.49,439358078,9.78E+12,eng,870,2153167,29221,09-01-04,Scholastic Inc.,NaN
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.78E+12,eng,352,6333,244,11-01-03,Scholastic,NaN
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPrÃ©,4.56,043965548X,9.78E+12,eng,435,2339585,36325,05-01-04,Scholastic Inc.,NaN
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPrÃ©,4.78,439682584,9.78E+12,eng,2690,41428,164,9/13/2004,Scholastic,NaN
5,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.74,976540606,9.78E+12,en-US,152,19,1,4/26/2005,Nimble Books,NaN
6,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,439827604,9.78E+12,eng,3342,28242,808,09-12-05,Scholastic,NaN
7,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,517226952,9.78E+12,eng,815,3628,254,11-01-05,Gramercy Books,NaN
8,13,The Ultimate Hitchhiker's Guide to the Galaxy ...,Douglas Adams,4.38,345453743,9.78E+12,eng,815,249558,4080,4/30/2002,Del Rey Books,NaN
9,14,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams,4.22,1400052920,9.78E+12,eng,215,4930,460,08-03-04,Crown,NaN


In [132]:
# Checking the datatypes
df.dtypes
# "average_rating" column seems to be object. Lets change it into float.
df["average_rating"] = df["average_rating"].astype(float)
# Checking again.
df.dtypes

ValueError: could not convert string to float: ' Jr./Sam B. Warner'

In [133]:
# Get the non-NA values in column "err"
non_na_values = df["err"].notna()
# Filter the DataFrame to get only the rows with non-NA values in column A
df_non_na = df[non_na_values]
df_non_na

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,err
3348,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner,Jr./Sam B. Warner,3.58,674842111,9.78E+12,en-US,236,61,6,4/20/2004,Harvard University Press
4702,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net,one of the founding members of this Tolkien w...,3.58,1593600119,9.78E+12,eng,400,26,4,04-06-04,Cold Spring Press
5877,22128,Patriots (The Coming Collapse),James Wesley,Rawles,3.63,156384155X,9.78E+12,eng,342,38,4,1/15/1999,Huntington House Publishers
8979,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown,Son & Ferguson,0,851742718,9.78E+12,eng,49,0,0,05-01-77,Brown Son & Ferguson Ltd.


In [134]:
# Making a list of indexes to drop the rows.
indexes_of_rows_to_drop = list(df_non_na.index)
indexes_of_rows_to_drop
# Dropping these rows
df = df.drop(index=[3348, 4702, 5877, 8979])

In [135]:
# Get the non-NA values in column "err"
non_na_values = df["err"].notna()
# Filter the DataFrame to get only the rows with non-NA values in column "err"
df_non_na = df[non_na_values]
df_non_na

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,err


In [136]:
# Checking "err" column again to see if there are any other values than NaN.
df["err"].isna().value_counts()
# Dropping "err" column because it is an artificial column I just created to import the dataframe.
df.drop(["err"], axis = 1, inplace = True)

In [167]:
# "isbn" column is just another unique number to identify the book, the International Standard Book Number.
# "isbn13" column is same but with 13 digits.
# I don't need other identification numbers for the books dataset so I drop these two columns.
df.drop(["isbn","isbn13"], axis = 1, inplace = True)
df.head()

,bookID,title,authors,average_rating,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPrÃ©,4.57,eng,652,2095690,27591,2006-09-16,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPrÃ©,4.49,eng,870,2153167,29221,2004-09-01,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,2003-11-01,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPrÃ©,4.56,eng,435,2339585,36325,2004-05-01,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPrÃ©,4.78,eng,2690,41428,164,2004-09-13,Scholastic


In [140]:
# Now able to change the datatype of "average_rating" to float.
df["average_rating"] = df["average_rating"].astype(np.float64)
# Changing the Number of Pages column "num_pages" to int.
df["num_pages"] = df["num_pages"].astype(np.int64)

In [141]:
# Seems like num_pages has some spaces in the column names. Lets strip all column names so we have no whitespaces before and after.
df.columns = df.columns.str.strip()

In [142]:
# Lets try again to change the datatype after stripping column names.
df["num_pages"] = df["num_pages"].astype(np.int64)

In [143]:
display(df.head(10))
df.dtypes

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPrÃ©,4.57,439785960,9.78E+12,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPrÃ©,4.49,439358078,9.78E+12,eng,870,2153167,29221,09-01-04,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.78E+12,eng,352,6333,244,11-01-03,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPrÃ©,4.56,043965548X,9.78E+12,eng,435,2339585,36325,05-01-04,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPrÃ©,4.78,439682584,9.78E+12,eng,2690,41428,164,9/13/2004,Scholastic
5,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.74,976540606,9.78E+12,en-US,152,19,1,4/26/2005,Nimble Books
6,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,439827604,9.78E+12,eng,3342,28242,808,09-12-05,Scholastic
7,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,517226952,9.78E+12,eng,815,3628,254,11-01-05,Gramercy Books
8,13,The Ultimate Hitchhiker's Guide to the Galaxy ...,Douglas Adams,4.38,345453743,9.78E+12,eng,815,249558,4080,4/30/2002,Del Rey Books
9,14,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams,4.22,1400052920,9.78E+12,eng,215,4930,460,08-03-04,Crown


bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                 object
language_code          object
num_pages               int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
dtype: object

In [144]:
# Lets check if all publication dates are in DD/MM/YYYY format to fix all of them.
df["publication_date"].tail(15)

11112      07-10-77
11113      06-01-04
11114      11-07-06
11115     4/16/2004
11116     7/17/2004
11117     8/15/2003
11118      04-08-03
11119      09-01-04
11120      10-11-03
11121      02-01-94
11122    12/21/2004
11123      12-01-88
11124      08-01-93
11125     2/27/2007
11126     5/28/2006
Name: publication_date, dtype: object

In [145]:
df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce')
df.tail(15)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
11112,45595,La TÃ­a Julia y el Escribidor,Mario Vargas Llosa,3.92,8432203238,9.79E+12,spa,447,25,4,1977-07-10,Seix Barral
11113,45604,Narraciones Extraordinarias,Edgar Allan Poe,4.13,9583006408,9.79E+12,spa,316,36,4,2004-06-01,Panamericana Editorial
11114,45607,Las CrÃ³nicas de Narnia,C.S. Lewis/Margarita E. Valdes/Gemma Gallart/P...,4.26,61199001,9.78E+12,spa,816,186,11,2006-11-07,HarperCollins Espanol
11115,45615,O Trono de Prata (As CrÃ³nicas de NÃ¡rnia #6),C.S. Lewis/Ana FalcÃ£o Bastos,3.96,972233168X,9.79E+12,por,168,141,10,2004-04-16,Editorial PresenÃ§a
11116,45616,A Ãltima Batalha (As CrÃ³nicas de NÃ¡rnia #7),C.S. Lewis/Pauline Baynes/Ana FalcÃ£o Bastos,4.03,9722332201,9.79E+12,por,149,211,24,2004-07-17,Editorial PresenÃ§a
11117,45617,O Cavalo e o Seu Rapaz (As CrÃ³nicas de NÃ¡rni...,C.S. Lewis/Pauline Baynes/Ana FalcÃ£o Bastos,3.92,9722330551,9.79E+12,por,160,207,16,2003-08-15,Editorial PresenÃ§a
11118,45623,O Sobrinho do MÃ¡gico (As CrÃ³nicas de NÃ¡rnia...,C.S. Lewis/Pauline Baynes/Ana FalcÃ£o Bastos,4.04,9722329987,9.79E+12,por,147,396,37,2003-04-08,Editorial PresenÃ§a
11119,45625,A Viagem do Caminheiro da Alvorada (As CrÃ³nic...,C.S. Lewis/Pauline Baynes/Ana FalcÃ£o Bastos,4.09,9722331329,9.79E+12,por,176,161,14,2004-09-01,Editorial PresenÃ§a
11120,45626,O PrÃ­ncipe Caspian (As CrÃ³nicas de NÃ¡rnia #4),C.S. Lewis/Pauline Baynes/Ana FalcÃ£o Bastos,3.97,9722330977,9.79E+12,por,160,215,11,2003-10-11,Editorial PresenÃ§a
11121,45630,Whores for Gloria,William T. Vollmann,3.69,140231579,9.78E+12,en-US,160,932,111,1994-02-01,Penguin Books


In [148]:
# Checking NaT (Not a Time) rows to fix.
invalid_rows = df['publication_date'].isna()
print(df.loc[invalid_rows, 'publication_date'])

8180    NaT
11098   NaT
Name: publication_date, dtype: datetime64[ns]


In [158]:
# Fixing two cells where the publication_date is corrupted.
df.loc[8180, 'publication_date'] = pd.to_datetime('1999-01-01')
df.loc[11098, 'publication_date'] = pd.to_datetime('1975-01-01')

In [168]:
# All datatypes are fixed now.
display(df.dtypes)
df.head()

bookID                         int64
title                         object
authors                       object
average_rating               float64
language_code                 object
num_pages                      int64
ratings_count                  int64
text_reviews_count             int64
publication_date      datetime64[ns]
publisher                     object
dtype: object

,bookID,title,authors,average_rating,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPrÃ©,4.57,eng,652,2095690,27591,2006-09-16,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPrÃ©,4.49,eng,870,2153167,29221,2004-09-01,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,2003-11-01,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPrÃ©,4.56,eng,435,2339585,36325,2004-05-01,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPrÃ©,4.78,eng,2690,41428,164,2004-09-13,Scholastic


In [169]:
# Checking for missing values in the dataframe.
df.isna().sum()
# No missing values are found.

bookID                0
title                 0
authors               0
average_rating        0
language_code         0
num_pages             0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64

In [90]:
df['language_code'].unique()

array(['eng', 'en-US', 'fre', 'spa', 'en-GB', 'mul', 'grc', 'enm',
       'en-CA', 'ger', 'jpn', 'ara', 'nl', 'zho', 'lat', 'por', 'srp',
       '9.78E+12', 'ita', 'rus', 'msa', 'glg', 'wel', 'swe', 'nor', 'tur',
       'gla', 'ale'], dtype=object)

In [173]:
df['language_code'].value_counts()

eng      8908
en-US    1408
spa       218
en-GB     214
fre       144
ger        99
jpn        46
mul        19
zho        14
grc        11
por        10
en-CA       7
ita         5
enm         3
lat         3
swe         2
rus         2
srp         1
nl          1
msa         1
glg         1
wel         1
ara         1
nor         1
tur         1
gla         1
ale         1
Name: language_code, dtype: int64

In [180]:
# Fixing the Language Codes and unifying them. Changing language codes to language names.
# Dictionary that consists Language codes to Language names.
lang_dict = {
    'eng': 'English',
    'en-US': 'English',
    'en-GB': 'English',
    'en-CA': 'English',
    'enm': 'Middle English',
    'fre': 'French',
    'ger': 'German',
    'spa': 'Spanish',
    'jpn': 'Japanese',
    'por': 'Portuguese',
    'lat': 'Latin',
    'grc': 'Ancient Greek',
    'ita': 'Italian',
    'zho': 'Chinese',
    'rus': 'Russian',
    'swe': 'Swedish',
    'msa': 'Malay',
    'nl': 'Dutch',
    'glg': 'Galician',
    'srp': 'Serbian',
    'tur': 'Turkish',
    'gla': 'Scottish Gaelic',
    'wel': 'Welsh',
    'ara': 'Arabic',
    'nor': 'Norwegian',
    'ale': 'Aleut',
    'mul': 'Multiple Languages'
}



# Replacing language codes with language names.
df['language_name'] = df['language_code'].replace(lang_dict)

# Dropping language code column.
df.drop(["language_code"], axis=1, inplace=True)

df['language_name'].value_counts()

English               10537
Spanish                 218
French                  144
German                   99
Japanese                 46
Multiple Languages       19
Chinese                  14
Ancient Greek            11
Portuguese               10
Italian                   5
Latin                     3
Middle English            3
Russian                   2
Swedish                   2
Dutch                     1
Arabic                    1
Serbian                   1
Malay                     1
Galician                  1
Welsh                     1
Norwegian                 1
Turkish                   1
Scottish Gaelic           1
Aleut                     1
Name: language_name, dtype: int64

In [182]:
df.head()

,bookID,title,authors,average_rating,num_pages,ratings_count,text_reviews_count,publication_date,publisher,language_name
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPrÃ©,4.57,652,2095690,27591,2006-09-16,Scholastic Inc.,English
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPrÃ©,4.49,870,2153167,29221,2004-09-01,Scholastic Inc.,English
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,352,6333,244,2003-11-01,Scholastic,English
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPrÃ©,4.56,435,2339585,36325,2004-05-01,Scholastic Inc.,English
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPrÃ©,4.78,2690,41428,164,2004-09-13,Scholastic,English


In [203]:
one_author_df = df.copy()

In [204]:
one_author_df["authors"]

0                              J.K. Rowling/Mary GrandPrÃ©
1                              J.K. Rowling/Mary GrandPrÃ©
2                                             J.K. Rowling
3                              J.K. Rowling/Mary GrandPrÃ©
4                              J.K. Rowling/Mary GrandPrÃ©
                               ...                        
11122    William T. Vollmann/Larry McCaffery/Michael He...
11123                                  William T. Vollmann
11124                                  William T. Vollmann
11125                                  William T. Vollmann
11126                                           Mark Twain
Name: authors, Length: 11123, dtype: object

In [215]:
# Checking if there are any duplicate rows.
df.duplicated().sum()

0

In [253]:
# Data validation on "publication_date"
# 91 publication dates has to be fixed, actually the dates are almost correct put the century has to be 20th century instead of the 21th century.
df[df["publication_date"] > "2021-01-01"]
# Subtract 100 years from dates greater than 2021-01-01 because they are supposed to be in 20th century.
df.loc[df['publication_date'] > '2021-01-01', 'publication_date'] -= pd.DateOffset(years=100)

In [260]:
def splitting_string(text):
    first_author = text.split('/')
    return first_author[0]

one_author_df['only_author'] = one_author_df['authors'].apply(lambda x : splitting_string(x))

one_author_df

,bookID,title,authors,average_rating,num_pages,ratings_count,text_reviews_count,publication_date,publisher,language_name,only_author
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPrÃ©,4.57,652,2095690,27591,2006-09-16,Scholastic Inc.,English,J.K. Rowling
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPrÃ©,4.49,870,2153167,29221,2004-09-01,Scholastic Inc.,English,J.K. Rowling
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,352,6333,244,2003-11-01,Scholastic,English,J.K. Rowling
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPrÃ©,4.56,435,2339585,36325,2004-05-01,Scholastic Inc.,English,J.K. Rowling
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPrÃ©,4.78,2690,41428,164,2004-09-13,Scholastic,English,J.K. Rowling
...,...,...,...,...,...,...,...,...,...,...,...
11122,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,512,156,20,2004-12-21,Da Capo Press,English,William T. Vollmann
11123,45633,You Bright and Risen Angels,William T. Vollmann,4.08,635,783,56,1988-12-01,Penguin Books,English,William T. Vollmann
11124,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,415,820,95,1993-08-01,Penguin Books,English,William T. Vollmann
11125,45639,Poor People,William T. Vollmann,3.72,434,769,139,2007-02-27,Ecco,English,William T. Vollmann


In [ ]:
#one_author_df[["title","only_author"]].loc[100:]
one_author_df["average_rating"].value_counts()
# I want to do EDA on these so I will keep. If I intend to remove, I can remove before my machine model.

In [8]:
import pickle
# Loading pickle file into dictionary.
with open('extract5.pkl', 'rb') as handle:
    loaded_dictionary = pickle.load(handle)
    print('dictionary loaded successfully to variable')

loaded_dictionary

dictionary loaded successfully to variable


{'30646': ['Psychology', 'Philosophy', 'Nonfiction'],
 '30659': ['Philosophy', 'Nonfiction', 'Classics'],
 '30666': ['Classics', 'Fiction', 'Fantasy'],
 '30671': ['Science', 'History', 'Nonfiction'],
 '30675': ['History', 'Nonfiction', 'Religion'],
 '30676': ['Psychology', 'Philosophy', 'Nonfiction'],
 '30677': ['Philosophy', 'Psychology', 'Nonfiction'],
 '30680': ['Fiction', 'Travel', 'Contemporary'],
 '30681': ['Fiction', 'Young Adult', 'Coming Of Age'],
 '30697': ['Realistic Fiction', 'Young Adult', 'Fiction'],
 '30702': ['Psychology', 'Self Help', 'Relationships'],
 '30712': ['Nonfiction', 'Essays', 'Books About Books'],
 '30713': ['Fiction', 'Short Stories', 'Classics'],
 '30733': ['Philosophy', 'Classics', 'Essays'],
 '30734': ['Philosophy', 'Essays', 'Classics'],
 '30735': ['Philosophy', 'Essays', 'Classics'],
 '30739': ['Philosophy', 'Nonfiction', 'Classics'],
 '30771': ['Manga', 'Horror', 'Vampires'],
 '30774': ['Manga', 'Horror', 'Vampires'],
 '30810': ['Nonfiction', 'Politic

# EDA (Explanatory Data Analysis)